# Using Qiskit Terra


Qiskit Terra contains tools that **define**, **compile** and **execute** quantum circuits on arbitrary **backends**.
It allows users to manipulate qubits by using gates and even construct circuits at the level of **pulses**

In [6]:
# Import required fuctions from qiskit
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, IBMQ
from qiskit.compiler import transpile, assemble
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-internal', group='support', project='core-team')
backend = provider.backends.ibmq_singapore

ibmqfactory.load_account:WARNING:2020-06-05 09:56:07,407: Credentials are already in use. The existing account in the session will be replaced.


# Transpiler Optimization levels 

optimization_level: How much optimization to perform on the circuits.
   - Higher levels generate more optimized circuits, at the expense of longer transpilation time.
            0: no optimization
            1: light optimization
            2: heavy optimization
            3: even heavier optimization
            If ``None``, level 1 will be chosen as default.




These optimization levels are provided via "preset passmanagers" in Qiskit. These are simple-to-use transpiler pipelines. You can see what each does by inspecting the documentation for those: https://github.com/Qiskit/qiskit-terra/tree/master/qiskit/transpiler/preset_passmanagers

Level 0 does no explicit optimization, it will just try to make a circuit runnable by transforming it to match a topology and basis gate set, if necessary.

Level 1, 2 and 3 do light, medium and heavy optimization, using a combination of passes, and by configuring the passes to search for better solutions. Typically more optimality comes at the cost of slower transpilation, as it spends more time searching for a good optimization.

The difference between level 2 and level 3: 
   - In level 2 the transpiler does some commutation analysis to see what gates can be collapsed. 
   - In level 3, in addition to that, it does peep-hole optimization by combining a chain of gates on the same qubits and re-synthesizing them with better cost. Also the routing stage of level 3 is more expensive than level 2 (it has a larger search space, as set by parameters of the StochasticSwap and LookaheadSwap passes).

It is important to keep in mind that these are heuristics. It is possible that, for a specific circuit, you will not see much improvement by increasing the level.


The optimization level that is chosen impacts the mapping strategies as follow : 
   - Trivial layout: Map virtual qubits to the same numbered physical qubit on the device, i.e. [0,1,2,3,4] -> [0,1,2,3,4] (default in optimization_level=0).

   - Dense layout: Find the sub-graph of the device with same number of qubits as the circuit with the greatest connectivity (default in optimization_level=1).

   - Noise adaptive layout: Uses the noise properties of the device, in concert with the circuit properties, to generate the layout with the best noise properties (default in optimization_level=2 and optimization_level=3).



In [79]:
# from Circuit Library 
# We Generate a random circuit of arbitrary size and form. 
# This function will generate a random circuit by randomly selecting gates from the set of standard gates 
from qiskit.circuit.random import random_circuit

circ = random_circuit(5, 5, measure=False)

print(circ.count_ops())

circ.draw(output='mpl')

In [80]:
# Let's transpile that circuit. We'll use simgapore as backend 
# backend: when set, the transpiler options are automatically grabbed from `backend.configuration()` 
# and `backend.properties()`. If any other option is explicitly set (e.g., ``coupling_map``), it 
# will override the backend's.

tpl_circ = transpile(circ, backend)

# count of operations
print(tpl_circ.count_ops())

# draw the circuit when transpiled
tpl_circ.draw(output='mpl', idle_wires=True)



## *Now it's your turn to play!*
<div class="alert alert-block alert-info">
<b>Note:</b>
Here's a first challenge for you:  
    
  - Generate a random circuit with 8 qubits and a depth of 10. Add the measure to the circuit
  - Tranpile that circuit using the backend 'ibmq_singapore' using different optimization_levels (0, 1, 2 and 3.
  - Draw each circuit
  - Count operations and depth of each transpiled circuits
 </div> 
</div>

In [ ]:
# Generate your random circuit 




In [ ]:
# transpile your circuit using ibmq_singapore as backend




In [ ]:
# print out the count operations 




In [ ]:
# draw the circuit 




<div class="alert alert-block alert-info">
<b>Note:</b>
Here's a second challenge for you: Consider the circuit "qc" below  
 
  - First transpile the circuit using optimization levels 1, 2 and 3
  - Use assemble to create a Qobj of the 3 circuits. 
  - Run that list of circuit in one job using "ibmq_singapore" as backend
  - Plot the counts results for each circuit   
 </div> 
</div>

In [81]:
from qiskit import QuantumCircuit

qc = QuantumCircuit(4,3)
qc.x(3)
qc.h(3)
qc.h(range(3))
qc.cx(range(3), 3)
qc.h(range(3))
qc.barrier(range(3))
qc.measure(range(3), range(3))
qc.draw(output='mpl')

In [ ]:
# qiskit import 
from qiskit.compiler import assemble, transpile


In [ ]:
# Set your backend to "ibmq_singapore" 


In [ ]:
# Transpile the "qc" circuit using the optimization levels 1, 2 and 3 


In [ ]:
# print the operations count for each circuit transpiled and its depth as well 


In [143]:
# Assemble the 3 circuits transpiled into one Qobj. 
# assemble function for converting a list of circuits into a qobj (pass a list of the 3 circuits)
# you must create a list containing the 3 circuits and pass the list to "assemble"
 


In [ ]:
# run the Qobj in one job using your backend 


In [ ]:
# get the count results of the job 


In [ ]:
# from the count results get the count for each circuit 


In [ ]:
# plot the counts for each circuit in one plot_histogram 


# Pass Manager 

In [89]:
# execute the code below using the 'qasm_simulator'
from qiskit import Aer
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import BasicSwap
from qiskit.compiler import transpile
from qiskit.transpiler import CouplingMap

coupling = [[0, 1], [1, 2], [2, 3], [3, 4], [4, 5], [5, 6]]


simulator = Aer.get_backend('qasm_simulator')


coupling_map = CouplingMap(couplinglist=coupling)


pm = PassManager()

pm.append([BasicSwap(coupling_map=coupling_map)])

# transpiled_circuit = pm.run(tpl_circuit)
tpl_circuit = pm.run(qc)

tpl_circuit.draw(output='mpl')



In [90]:
# execute the same code on a real backend 'ibmq_16_melbourne'
realdevice = provider.backends.ibmq_16_melbourne
tpl_realdevice = transpile(tpl_circuit, backend = realdevice, optimization_level=2)
tpl_realdevice.draw(output='mpl', idle_wires=False)

<div class="alert alert-block alert-info">
<b>Note:</b>
Here's a question for you: Consider the circuit tpl_realdevice 
 
  - How do you explain the difference in the number of CNOT gates between the circuit transpiled on the simulator and the circuit transpiled on the real backend 'ibmq_16_melbourne' 
 </div> 
</div>